In [82]:
!pip install flat-table

import pandas as pd
import json 
from pandas.io.json import json_normalize #package for flattening json in pandas df
import flat_table as ft #another package for flattening json in pandas df that also handles nulls nicely
import math
import numpy as np

In [83]:
yelp = pd.read_json("yelp_academic_dataset_business.json", lines= True)
yelp.info()
yelp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209393 entries, 0 to 209392
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   209393 non-null  object 
 1   name          209393 non-null  object 
 2   address       209393 non-null  object 
 3   city          209393 non-null  object 
 4   state         209393 non-null  object 
 5   postal_code   209393 non-null  object 
 6   latitude      209393 non-null  float64
 7   longitude     209393 non-null  float64
 8   stars         209393 non-null  float64
 9   review_count  209393 non-null  int64  
 10  is_open       209393 non-null  int64  
 11  attributes    180348 non-null  object 
 12  categories    208869 non-null  object 
 13  hours         164550 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 22.4+ MB


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [84]:
# ensure business_id is unique
yelp['business_id'].duplicated().sum()

0

In [85]:
#Check for nulls
yelp.isna().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      29045
categories        524
hours           44843
dtype: int64

# Removing and Cleaning Data

Looking at the value counts of our business data per state, it is clear that our data is only representative of a few states. 

In [86]:
yelp['state'].value_counts()

AZ     60803
NV     39084
ON     36627
OH     16392
NC     16218
PA     12376
QC     10233
AB      8682
WI      5525
IL      2034
SC      1328
CA        23
NY        22
TX         6
WA         5
FL         3
GA         3
AL         3
BC         2
CO         2
UT         2
NE         2
CT         2
MI         2
VT         2
MO         1
AR         1
MB         1
OR         1
AK         1
HI         1
HPL        1
YT         1
DOW        1
VA         1
DUR        1
XWY        1
Name: state, dtype: int64

This is unfortunate, but there's nothing we can really do aside for looking for other similar datasets to combine with this one (which we tried, and unfortunately came up short). However, interestingly enough we have data from a lot of states where we only have at most a couple dozen businesses. While this might not seem like a problem, it can actually make things a bit misleading as we continue with the project. Since all of our queries for our business data involve taking aggregates, it is quite misleading to compare our Arizona aggregate (with 60803 businesses) with, say, our California aggregate (with 23 businesses). We could always mention the denominator on the app itself, but it does not seem like responsible data science to show our users data and hope they are savvy enough to read a disclaimer and not use it. It is more responsible, then, to simply not show business data about states with very low representation (CA and lower). If we get rid of these values now, it will help us avoid mistakes when querying later on, as if there is unrepresentative business data, it will simply be null, as opposed to giving us some aggregate numbers that we then forget are only aggregated over 10 samples, etc.

All this is to say, let's just be safe and get rid of all the yelp data with state values of 23 and lower now.

We should also remove any records of businesses in Canada, since the other datasets we are using all have to do with the US exclusively.

Note: originally, I was going to do this part last, but after realizing that some of my decisions for preprocessing JSON would come from visualizations and null-counts, and since we decided that the aforementioned regions are not useful to us, I decided to do it first so that our preprocessing visualizations are more accurate.

In [87]:
#store a copy of the dataset
yelp_full = yelp.copy()

In [88]:
#get drop indices for states with value counts under 1000
drop_indices = yelp[yelp['state'].map(yelp['state'].value_counts() <1000)].index

yelp_reduced= yelp.drop(drop_indices, inplace=False)

#now get drop indices for Canada regions
drop_indices = yelp_reduced[yelp_reduced['state'].isin(['ON', 'QC', 'AB'])].index

yelp_reduced.drop(drop_indices, inplace=True)
yelp_reduced.reset_index(drop=True, inplace=True)

yelp_reduced['state'].value_counts()

AZ    60803
NV    39084
OH    16392
NC    16218
PA    12376
WI     5525
IL     2034
SC     1328
Name: state, dtype: int64

In [89]:
yelp_reduced.info()
yelp_reduced.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153760 entries, 0 to 153759
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   153760 non-null  object 
 1   name          153760 non-null  object 
 2   address       153760 non-null  object 
 3   city          153760 non-null  object 
 4   state         153760 non-null  object 
 5   postal_code   153760 non-null  object 
 6   latitude      153760 non-null  float64
 7   longitude     153760 non-null  float64
 8   stars         153760 non-null  float64
 9   review_count  153760 non-null  int64  
 10  is_open       153760 non-null  int64  
 11  attributes    133132 non-null  object 
 12  categories    153367 non-null  object 
 13  hours         123204 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.4+ MB


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
3,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."
4,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",Gilbert,AZ,85233,33.350399,-111.827142,4.5,38,1,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Automotive, Oil Change Stations, ...","{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ..."


In [90]:
yelp_reduced['is_open'].value_counts()

1    125135
0     28625
Name: is_open, dtype: int64

In [91]:
#get rid of closed businesses, as we have no need for them
drop_indices = yelp_reduced[yelp_reduced['is_open'] == 0].index

yelp_reduced.drop(drop_indices, inplace=True)

#get rid of address and is_open, as we don't really need them
yelp_reduced.drop('address', inplace=True, axis=1)
yelp_reduced.drop('is_open', inplace=True, axis=1)

yelp_reduced.reset_index(drop=True, inplace=True)


yelp_reduced.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125135 entries, 0 to 125134
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   125135 non-null  object 
 1   name          125135 non-null  object 
 2   city          125135 non-null  object 
 3   state         125135 non-null  object 
 4   postal_code   125135 non-null  object 
 5   latitude      125135 non-null  float64
 6   longitude     125135 non-null  float64
 7   stars         125135 non-null  float64
 8   review_count  125135 non-null  int64  
 9   attributes    107212 non-null  object 
 10  categories    124761 non-null  object 
 11  hours         103877 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 11.5+ MB


In [92]:
yelp_reduced.head()

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."
3,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,Gilbert,AZ,85233,33.350399,-111.827142,4.5,38,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Automotive, Oil Change Stations, ...","{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ..."
4,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,Las Vegas,NV,89118,36.063977,-115.241463,3.5,81,"{'BusinessParking': '{'garage': False, 'street...","Dry Cleaning & Laundry, Local Services, Laundr...","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ..."


In [93]:
#Now check to make sure city doesn't have any weird formatting going on. We can look for punctuation to help find out.
yelp_reduced[yelp_reduced['city'].str.contains('[^A-Za-z0-9\s]', regex = True )]

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
703,i9PUoStRuesSnIKWY6ZOrw,Wicked Pixie Salon & Hair Studio,​Gilbert,AZ,85297,33.276765,-111.786945,5.0,17,"{'GoodForKids': 'True', 'RestaurantsPriceRange...","Hair Extensions, Hair Salons, Hair Removal, Bl...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-19:0'..."
1265,Y28XY3fko-Sdb9RBBgCDRQ,DiBella's Subs,Warrensville Hts.,OH,44122,41.447229,-81.496798,3.5,47,"{'RestaurantsReservations': 'False', 'Alcohol'...","Restaurants, Delis, Salad, Sandwiches","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
1320,squM96Zm0Ae6K3wvaHLuBw,Attention To Detail - Car Care Specialists,"Urbana, Illinois",IL,61802,40.115516,-88.202043,5.0,20,"{'ByAppointmentOnly': 'False', 'BusinessAccept...","Automotive, Auto Detailing, Car Wash","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4744,dXzLioqrEncX5nONdhhjMg,Anderson's Sports Pub & Eatery,Mentor-on-the-Lake,OH,44060,41.706009,-81.361561,3.0,16,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Sports Bars, Irish, Nightlife, Restaurants, Bars","{'Monday': '11:0-0:0', 'Tuesday': '11:0-0:0', ..."
6124,sCLXU9ehXMNsodcZqXKNZA,King China Buffet,Mentor-on-the-Lake,OH,44060,41.707105,-81.361282,3.0,5,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Restaurants, Chinese","{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3..."
...,...,...,...,...,...,...,...,...,...,...,...,...
117308,y4giWpQy1TYYX7sZOk9NNg,Wendy's,Middleburg Hts.,OH,44130,41.369281,-81.804608,1.0,17,"{'GoodForMeal': '{'dessert': False, 'latenight...","Fast Food, Restaurants, Burgers, Hot Dogs","{'Monday': '10:0-1:0', 'Tuesday': '10:0-1:0', ..."
117639,bAqt_TWv-f42GWCG2h0_8Q,Cookies by Design,"North Olmsted,",OH,44070,41.426557,-81.893084,5.0,10,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Bakeries, Flowers & Gifts, Shopping, Food, Des...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."
119535,DtrTCViUn9uV8TtD_eyMQg,Ambiance - The Store For Lovers,Mayfield Hts.,OH,44124,41.520459,-81.467055,3.0,4,"{'BusinessParking': '{'garage': False, 'street...","Adult, Personal Shopping, Lingerie, Shopping, ...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-21:0', ..."
121367,TYtyWZOtKNQ1yQS1HJUApw,Mo & Co Hair Factory,"Las Vegas, Nevada",NV,89120,36.071657,-115.094138,4.5,9,"{'ByAppointmentOnly': 'False', 'GoodForKids': ...","Beauty & Spas, Barbers, Hair Salons, Hair Exte...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."


We see that some of our city values do contain punctuation. Some of these seem fine (periods for abbreviatons, dashes for certain hyphenated names, etc.), but the commas are interesting. It seems some of our cities also have an unneccessary state attached. Let's fix that.

In [94]:
yelp_reduced['city'] = yelp_reduced['city'].apply(lambda x: x.split(',')[0])

yelp_reduced[yelp_reduced['city'].str.contains(',')]

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours


In [95]:
#Check for nulls again
yelp_reduced.isna().sum()

business_id         0
name                0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
attributes      17923
categories        374
hours           21258
dtype: int64

## Zipcode entity Resolution

In [96]:
zips = pd.read_csv('zip_data.csv')
zips

,zip,type,decommissioned,primary_city,state,county,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NY,Suffolk County,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NY,Suffolk County,US,40.81,-73.04,0
2,1001,STANDARD,0,Agawam,MA,Hampden County,US,42.06,-72.61,15220
3,1002,STANDARD,0,Amherst,MA,Hampshire County,US,42.37,-72.52,16570
4,1003,PO BOX,0,Amherst,MA,Hampshire County,US,42.39,-72.52,184
...,...,...,...,...,...,...,...,...,...,...
41696,99926,PO BOX,0,Metlakatla,AK,Prince Of Wales-Hyder Census Area,US,55.14,-131.49,1140
41697,99927,PO BOX,0,Point Baker,AK,Prince Of Wales-Hyder Census Area,US,56.30,-133.57,48
41698,99928,PO BOX,0,Ward Cove,AK,Ketchikan Gateway Borough,US,55.45,-131.79,1530
41699,99929,PO BOX,0,Wrangell,AK,Wrangell City And Borough,US,56.41,-131.61,2145


In [97]:
type(zips['zip'][0])

numpy.int64

In [98]:
type(yelp_reduced['postal_code'][0])

str

In [99]:
# We have spome blank values that are also not nulls that will make type conversion difficult.
yelp_reduced[yelp_reduced['postal_code'] == '']

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
751,gTn_YaXUYEC5py8ESrX5Wg,Maki Taco,Charlotte,NC,,35.227087,-80.843127,3.0,40,"{'RestaurantsPriceRange2': '1', 'RestaurantsGo...","Food Stands, Street Vendors, Restaurants, Mexi...","{'Monday': '11:0-14:0', 'Wednesday': '11:0-14:..."
904,dWapY1yn-vG_ECu67AIEKg,Bebo's Mac Shack,Charlotte,NC,,35.227087,-80.843127,2.5,7,"{'RestaurantsPriceRange2': '1', 'Caters': 'Tru...","Food, Burgers, Food Stands, Food Trucks, Resta...","{'Monday': '9:0-9:0', 'Tuesday': '9:0-9:0', 'W..."
1329,EIo9thPJJ86soQdnpyK7EA,SuperFarm SuperTruck,Mesa,AZ,,33.350782,-111.653307,3.5,30,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Food, Street Vendors",None
1749,RPpHS-I4Xf-IbQWb-iSDvA,Beckham's B&M Bar-B-Que,Cleveland,OH,,41.499320,-81.694361,4.0,4,"{'Caters': 'True', 'RestaurantsDelivery': 'Fal...","Food, Food Trucks",None
2502,EeOrrSmqNIoVnLW9kjEfOg,Scottsdale Neighborhood Trolley,Scottsdale,AZ,,33.494170,-111.926052,5.0,5,None,"Transportation, Public Transportation, Hotels ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...
123387,ofqNprsSZVhx9Hq2wRoLWA,Patrick Burt Sculpted Jewelry Designs,Tempe,AZ,,33.380022,-111.941030,5.0,3,"{'BusinessParking': '{'garage': False, 'street...","Local Flavor, Shopping, Local Services, Jewelr...",None
123598,7s43BalhWiHze8ckwZpegA,Noca Wich,Central,AZ,,33.434162,-111.996949,2.0,39,"{'GoodForMeal': '{'dessert': False, 'latenight...","Restaurants, Mexican, Sandwiches",None
123751,tseZGhzyTJgP_zJSqjzAFQ,Homeward Bound Cat Adoptions,Las Vegas,NV,,36.175000,-115.136389,4.0,10,None,"Pets, Animal Shelters","{'Thursday': '15:0-18:0', 'Friday': '15:0-18:0..."
123825,ebBjWSi7JQykAB05dykqGA,Envoy America,Phoenix,AZ,,33.448377,-112.074037,5.0,4,None,"Medical Transportation, Health & Medical, Loca...",None


In [100]:
#Convert blank string values to nulls
yelp_reduced.replace('', np.nan, inplace=True)
yelp_reduced.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125135 entries, 0 to 125134
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   125135 non-null  object 
 1   name          125134 non-null  object 
 2   city          125134 non-null  object 
 3   state         125135 non-null  object 
 4   postal_code   124921 non-null  object 
 5   latitude      125135 non-null  float64
 6   longitude     125135 non-null  float64
 7   stars         125135 non-null  float64
 8   review_count  125135 non-null  int64  
 9   attributes    107212 non-null  object 
 10  categories    124761 non-null  object 
 11  hours         103877 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 11.5+ MB


Seems yelp messed up and accidentally set a Toronto business to be in OH instead of ON. Let's drop that.

In [101]:
yelp_reduced[yelp_reduced['postal_code'].str.isnumeric() ==False]

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
82403,wfe31gLa1qtAgiCVI0cH2g,Pastucci's,Toronto,OH,M8Z 1N1,43.625879,-79.504026,4.0,25,"{'BusinessAcceptsBitcoin': 'False', 'DogsAllow...","Restaurants, Italian, Food, Specialty Food, Sa...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ..."


In [102]:
drop_index = yelp_reduced[yelp_reduced['postal_code'].str.isnumeric() ==False].index
yelp_reduced.drop(drop_index, inplace =True)
yelp_reduced.reset_index(drop=True, inplace=True)

yelp_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125134 entries, 0 to 125133
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   125134 non-null  object 
 1   name          125133 non-null  object 
 2   city          125133 non-null  object 
 3   state         125134 non-null  object 
 4   postal_code   124920 non-null  object 
 5   latitude      125134 non-null  float64
 6   longitude     125134 non-null  float64
 7   stars         125134 non-null  float64
 8   review_count  125134 non-null  int64  
 9   attributes    107211 non-null  object 
 10  categories    124760 non-null  object 
 11  hours         103876 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 11.5+ MB


In [103]:
yelp_reduced['postal_code'] = yelp_reduced['postal_code'].apply(lambda x: int(x) if type(x) == str  else x)

In [104]:
yelp_reduced.max()

business_id     zzwicjPC9g246MK2M1ZFBA
state                               WI
postal_code                     891118
latitude                       43.7519
longitude                      -76.804
stars                                5
review_count                     10129
dtype: object

In [105]:
yelp_reduced.min()

business_id     --7zmmkVg-IMGaXbuVd0SQ
state                               AZ
postal_code                       2645
latitude                       33.1035
longitude                     -115.493
stars                                1
review_count                         3
dtype: object

In [106]:
zips.max()

zip                               99950
type                             UNIQUE
decommissioned                        1
primary_city                     Zwolle
state                                WY
country                              US
latitude                          71.28
longitude                         33.53
irs_estimated_population_2015    114420
dtype: object

In [107]:
zips.min()

zip                                     501
type                               MILITARY
decommissioned                            0
primary_city                     Aaronsburg
state                                    AE
country                                  IQ
latitude                             -44.25
longitude                           -176.63
irs_estimated_population_2015             0
dtype: object

In [108]:
#Check if we have any yelp zip codes not included in our zips file

yelp_reduced[~yelp_reduced['postal_code'].isin(zips['zip'])]['postal_code'].value_counts()

85384.0     2
891118.0    1
44245.0     1
84224.0     1
85474.0     1
82595.0     1
44034.0     1
28225.0     1
35715.0     1
282010.0    1
85831.0     1
27273.0     1
89417.0     1
28251.0     1
88979.0     1
89184.0     1
15109.0     1
85394.0     1
88147.0     1
43714.0     1
44219.0     1
86006.0     1
71863.0     1
65241.0     1
85896.0     1
85052.0     1
15804.0     1
Name: postal_code, dtype: int64

In [109]:

yelp_reduced[~yelp_reduced['postal_code'].isin(zips['zip'])]

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
751,gTn_YaXUYEC5py8ESrX5Wg,Maki Taco,Charlotte,NC,NaN,35.227087,-80.843127,3.0,40,"{'RestaurantsPriceRange2': '1', 'RestaurantsGo...","Food Stands, Street Vendors, Restaurants, Mexi...","{'Monday': '11:0-14:0', 'Wednesday': '11:0-14:..."
904,dWapY1yn-vG_ECu67AIEKg,Bebo's Mac Shack,Charlotte,NC,NaN,35.227087,-80.843127,2.5,7,"{'RestaurantsPriceRange2': '1', 'Caters': 'Tru...","Food, Burgers, Food Stands, Food Trucks, Resta...","{'Monday': '9:0-9:0', 'Tuesday': '9:0-9:0', 'W..."
1329,EIo9thPJJ86soQdnpyK7EA,SuperFarm SuperTruck,Mesa,AZ,NaN,33.350782,-111.653307,3.5,30,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Food, Street Vendors",None
1749,RPpHS-I4Xf-IbQWb-iSDvA,Beckham's B&M Bar-B-Que,Cleveland,OH,NaN,41.499320,-81.694361,4.0,4,"{'Caters': 'True', 'RestaurantsDelivery': 'Fal...","Food, Food Trucks",None
2502,EeOrrSmqNIoVnLW9kjEfOg,Scottsdale Neighborhood Trolley,Scottsdale,AZ,NaN,33.494170,-111.926052,5.0,5,None,"Transportation, Public Transportation, Hotels ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...
123597,7s43BalhWiHze8ckwZpegA,Noca Wich,Central,AZ,NaN,33.434162,-111.996949,2.0,39,"{'GoodForMeal': '{'dessert': False, 'latenight...","Restaurants, Mexican, Sandwiches",None
123750,tseZGhzyTJgP_zJSqjzAFQ,Homeward Bound Cat Adoptions,Las Vegas,NV,NaN,36.175000,-115.136389,4.0,10,None,"Pets, Animal Shelters","{'Thursday': '15:0-18:0', 'Friday': '15:0-18:0..."
123824,ebBjWSi7JQykAB05dykqGA,Envoy America,Phoenix,AZ,NaN,33.448377,-112.074037,5.0,4,None,"Medical Transportation, Health & Medical, Loca...",None
123848,m1FVKF2easFE23iziLPThA,Pulp Juice And Smoothie Bar,Solon,OH,44219.0,41.383761,-81.435060,3.5,9,"{'GoodForMeal': '{'dessert': False, 'latenight...","Food, Juice Bars & Smoothies, Salad, Sandwiche...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ..."


There aren't many businesses not represented in our zipcode table, which is good! Let's get rid of the few that aren't so that we don't have any referential integrity issues making postal_code a foreign key to zip

In [110]:
drop_indices = yelp_reduced[~yelp_reduced['postal_code'].isin(zips['zip'])].index

yelp_reduced.drop(drop_indices, inplace=True)

yelp_reduced.reset_index(drop=True, inplace=True)


yelp_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124892 entries, 0 to 124891
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   124892 non-null  object 
 1   name          124891 non-null  object 
 2   city          124892 non-null  object 
 3   state         124892 non-null  object 
 4   postal_code   124892 non-null  float64
 5   latitude      124892 non-null  float64
 6   longitude     124892 non-null  float64
 7   stars         124892 non-null  float64
 8   review_count  124892 non-null  int64  
 9   attributes    107064 non-null  object 
 10  categories    124518 non-null  object 
 11  hours         103794 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 11.4+ MB


In [111]:
yelp_reduced[~yelp_reduced['postal_code'].isin(zips['zip'])]

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours


This last part is out of order, since the way I dsicovered it was a bit down the line, but we have to clean it here before we split tables.

In [33]:
yelp_flat[yelp_flat['name'].isna()]

,index,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.GoodForKids,attributes.BikeParking,attributes.BusinessAcceptsCreditCards,hours.Sunday,hours.Saturday,hours.Friday,hours.Thursday,hours.Wednesday,hours.Tuesday,hours.Monday
29837,29837,LiQckUkfrmaU09DijHLLpQ,NaN,Huntersville,NC,28078.0,35.383981,-80.786123,2.0,3,...,True,NaN,True,NaN,9:30-12:30,10:0-20:30,10:0-20:30,10:0-20:30,10:0-20:30,10:0-20:30


In [34]:
#What was that NaN originally
yelp[yelp['business_id'] == 'LiQckUkfrmaU09DijHLLpQ']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
49808,LiQckUkfrmaU09DijHLLpQ,,13024 Eastfield Rd,Huntersville,NC,28078,35.383981,-80.786123,2.0,3,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Martial Arts, Fitness & Instruction, Active Li...","{'Monday': '10:0-20:30', 'Tuesday': '10:0-20:3..."


Interestingly enough, flattening the table also seemed to convert a blank name to NaN for us. Technically, not having a listed name doesn't mean the other data is necessarily wrong, but it is certainly suspect. Ultimately, it's only one value, and having a null value in the column might make various type-based operations on the whole column more awkward, so let's remove it to be safe.

In [35]:
drop_index = yelp_reduced[yelp_reduced['name'].isna()].index

yelp_reduced.drop(drop_index, inplace = True)
yelp_reduced.reset_index(drop=True, inplace=True)
yelp_reduced.head()

,index,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.GoodForKids,attributes.BikeParking,attributes.BusinessAcceptsCreditCards,hours.Sunday,hours.Saturday,hours.Friday,hours.Thursday,hours.Wednesday,hours.Tuesday,hours.Monday
0,0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,Cornelius,NC,28031.0,35.462724,-80.852612,3.5,36,...,False,True,True,13:0-18:0,11:0-20:0,11:0-20:0,11:0-20:0,10:0-18:0,11:0-20:0,10:0-18:0
1,1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",Scottsdale,AZ,85258.0,33.569404,-111.890264,5.0,4,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,85205.0,33.428065,-111.726648,4.5,26,...,NaN,NaN,True,NaN,NaN,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,0:0-0:0
3,3,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,Gilbert,AZ,85233.0,33.350399,-111.827142,4.5,38,...,NaN,NaN,True,NaN,7:0-15:0,7:0-18:0,7:0-18:0,7:0-18:0,7:0-18:0,7:0-18:0
4,4,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,Las Vegas,NV,89118.0,36.063977,-115.241463,3.5,81,...,NaN,True,True,NaN,9:0-17:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0


In [36]:
yelp_reduced.drop('index', axis=1, inplace=True)

In [37]:
yelp_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124891 entries, 0 to 124890
Data columns (total 55 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   business_id                            124891 non-null  object 
 1   name                                   124891 non-null  object 
 2   city                                   124891 non-null  object 
 3   state                                  124891 non-null  object 
 4   postal_code                            124891 non-null  float64
 5   latitude                               124891 non-null  float64
 6   longitude                              124891 non-null  float64
 7   stars                                  124891 non-null  float64
 8   review_count                           124891 non-null  int64  
 9   attributes.Open24Hours                 11 non-null      object 
 10  attributes.DietaryRestrictions         22 non-null      

# Dealing With JSON

The main preprocessing step for our yelp data is converting our three json columns into a relational database. We have two main options here:

Option 1: Keep the json columns as is and store them in a NoSql manner (indeed, MySQL does support JSON fields, so we wouldn't necessarily have to even a use a "true" NoSQL database system). However, since we cannot index a JSON field, this may become quite difficult to optimize, and considering we will likely use these columns quite a bit, that might not be the best plan.

Option 2: Make our JSON into a more relational scheme by either expanding them into columns or exploding the values into new rows and breaking the corresponding column into a separate table with a many-to-one relationship with each business. This is a good deal more work, and means more joining in our queries, but I believe it will be more optimal and so is worth it.

We'll go with option 2!

Our JSON fields all contain different type of data:

The attributes field mainly contains dictionaries with True/False values (though it does contain some nested dictionaries). This is likely best dealt with by simply expanding these values into columns of the table, since we'll likely be searching on those particular values, and compiling a list of them as we would tags does not really make sense. The main consideration here is making sure all the key value pairings at all levels of the JSON hierarchy fall under the same true/false schema, and deciding how to deal if not. Otherwise, it's just a matter of making sure we fully expand each dictionary until our schema is in 1NF (whether we do this manually or recursively).

Hours contains a dictionary of days and time ranges (represented as strings). This also likely maps best to just expanding the columns, though we may have to get a bit fancy with our queries. The main considerations here are if we should convert the time strings to date-times, and also if we should store the start and end values for each time separately (or separately as well as in the original ranges), which would add a lot of extra overhead and/or space needed if we only really need time ranges, but could allow us to avoid costly regex operations in our queries if we want to frequently reference when things close and/or open irrespective to the other.

The categories column contains tags that apply to the business. While technically we could one-hot encode these tags in a similar manner to the attributes, that makes querying the list of tags that belong to a business quite awkward, and also wastes a lot of space with "False" values. Instead, we will explode the tags into multiple rows and break the column into its own table with a many-to-one relationship with businesses (for normal form purposes). But first, we need to "tokenize" our category values, which are currently lists written out as strings instead of actual lists. The other thing we need to consider is if there are potentially slightly different spellings for the same tag, in which case we will likely need to do some sort of jaccard join.

In [112]:
#Break categories off into its own table

categories = yelp_reduced[['business_id', 'categories']]
yelp_reduced = yelp_reduced.drop(['categories'], axis = 1)

# No need to keep null values for categories as simply not being in the list tells us if a tag is not included
categories.dropna(inplace=True)

First, let's take care of Attributes and Hours.

In [32]:
yelp_flat = ft.normalize(yelp_reduced)
yelp_flat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124892 entries, 0 to 124891
Data columns (total 56 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   index                                  124892 non-null  int64  
 1   business_id                            124892 non-null  object 
 2   name                                   124891 non-null  object 
 3   city                                   124892 non-null  object 
 4   state                                  124892 non-null  object 
 5   postal_code                            124892 non-null  float64
 6   latitude                               124892 non-null  float64
 7   longitude                              124892 non-null  float64
 8   stars                                  124892 non-null  float64
 9   review_count                           124892 non-null  int64  
 10  attributes.Open24Hours                 11 non-null      

## Hours

For hours, if we simply normalize without doing anything with the null values, we get an ambiguous situation where NaN represents both businesses that are not open on that day as well as businesses that contained no hour information at all. This is not good- we want to replace the NaNs in the former situation with something like a 'Closed' string instead. We could do this by looking at each column and changing the NaN value if any of the other hour columns are not null.

In [38]:
def null_to_closed(series, thresh, replacement_string):
    # if at least one of the hours columns has a non null value, fill all the hour nulls with "Closed"
    if series.isna().sum() < thresh:
        series.fillna(replacement_string, inplace = True)
    
    return series


In [39]:

hours = yelp_flat[['hours.Sunday','hours.Saturday', 'hours.Monday', 'hours.Tuesday', 'hours.Wednesday', 'hours.Thursday', 'hours.Friday']]
#7 hours columns, so set thresh at 7
hours = hours.apply(null_to_closed, args=(7, "Closed"), axis =1)


It will likely be more useful to us to have open and close times indiviudally rather than as range-denoting strings, so let's expand each hour column into two. I'm not converting into datetimes here as we also have strings- it may be better to convert within the SQL query

In [40]:
hours['hours.Sunday_open'] = hours['hours.Sunday'].apply(lambda x: x.split('-')[0] if (type(x) == str and x != 'Closed') else x)
hours['hours.Sunday_closed'] = hours['hours.Sunday'].apply(lambda x: x.split('-')[1] if (type(x) == str and x != 'Closed') else x)
hours['hours.Saturday_open'] = hours['hours.Saturday'].apply(lambda x: x.split('-')[0] if (type(x) == str and x != 'Closed') else x)
hours['hours.Saturday_closed'] = hours['hours.Saturday'].apply(lambda x: x.split('-')[1] if (type(x) == str and x != 'Closed') else x)
hours['hours.Monday_open'] = hours['hours.Monday'].apply(lambda x: x.split('-')[0] if (type(x) == str and x != 'Closed') else x)
hours['hours.Monday_closed'] = hours['hours.Monday'].apply(lambda x: x.split('-')[1] if (type(x) == str and x != 'Closed') else x)
hours['hours.Tuesday_open'] = hours['hours.Tuesday'].apply(lambda x: x.split('-')[0] if (type(x) == str and x != 'Closed') else x)
hours['hours.Tuesday_closed'] = hours['hours.Tuesday'].apply(lambda x: x.split('-')[1] if (type(x) == str and x != 'Closed') else x)
hours['hours.Wednesday_open'] = hours['hours.Wednesday'].apply(lambda x: x.split('-')[0] if (type(x) == str and x != 'Closed') else x)
hours['hours.Wednesday_closed'] = hours['hours.Wednesday'].apply(lambda x: x.split('-')[1] if (type(x) == str and x != 'Closed') else x)
hours['hours.Thursday_open'] = hours['hours.Thursday'].apply(lambda x: x.split('-')[0] if (type(x) == str and x != 'Closed') else x)
hours['hours.Thursday_closed'] = hours['hours.Thursday'].apply(lambda x: x.split('-')[1] if (type(x) == str and x != 'Closed') else x)
hours['hours.Friday_open'] = hours['hours.Friday'].apply(lambda x: x.split('-')[0] if (type(x) == str and x != 'Closed') else x)
hours['hours.Friday_closed'] = hours['hours.Friday'].apply(lambda x: x.split('-')[1] if (type(x) == str and x != 'Closed') else x)
hours.head()

,hours.Sunday,hours.Saturday,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Sunday_open,hours.Sunday_closed,hours.Saturday_open,...,hours.Monday_open,hours.Monday_closed,hours.Tuesday_open,hours.Tuesday_closed,hours.Wednesday_open,hours.Wednesday_closed,hours.Thursday_open,hours.Thursday_closed,hours.Friday_open,hours.Friday_closed
0,13:0-18:0,11:0-20:0,10:0-18:0,11:0-20:0,10:0-18:0,11:0-20:0,11:0-20:0,13:0,18:0,11:0,...,10:0,18:0,11:0,20:0,10:0,18:0,11:0,20:0,11:0,20:0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Closed,Closed,0:0-0:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,Closed,Closed,Closed,...,0:0,0:0,9:0,16:0,9:0,16:0,9:0,16:0,9:0,16:0
3,Closed,7:0-15:0,7:0-18:0,7:0-18:0,7:0-18:0,7:0-18:0,7:0-18:0,Closed,Closed,7:0,...,7:0,18:0,7:0,18:0,7:0,18:0,7:0,18:0,7:0,18:0
4,Closed,9:0-17:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0,Closed,Closed,9:0,...,7:0,19:0,7:0,19:0,7:0,19:0,7:0,19:0,7:0,19:0


In [41]:
yelp_flat.drop(['hours.Sunday','hours.Saturday', 'hours.Friday', 'hours.Thursday', 'hours.Wednesday', 'hours.Tuesday', 'hours.Monday'], axis=1, inplace=True)
yelp_flat = pd.concat([yelp_flat, hours], join='inner', axis=1)
yelp_flat.drop(['hours.Sunday','hours.Saturday', 'hours.Friday', 'hours.Thursday', 'hours.Wednesday', 'hours.Tuesday', 'hours.Monday'], axis=1, inplace=True)

yelp_flat.head()

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes.Open24Hours,...,hours.Monday_open,hours.Monday_closed,hours.Tuesday_open,hours.Tuesday_closed,hours.Wednesday_open,hours.Wednesday_closed,hours.Thursday_open,hours.Thursday_closed,hours.Friday_open,hours.Friday_closed
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,Cornelius,NC,28031.0,35.462724,-80.852612,3.5,36,NaN,...,10:0,18:0,11:0,20:0,10:0,18:0,11:0,20:0,11:0,20:0
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",Scottsdale,AZ,85258.0,33.569404,-111.890264,5.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,85205.0,33.428065,-111.726648,4.5,26,NaN,...,0:0,0:0,9:0,16:0,9:0,16:0,9:0,16:0,9:0,16:0
3,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,Gilbert,AZ,85233.0,33.350399,-111.827142,4.5,38,NaN,...,7:0,18:0,7:0,18:0,7:0,18:0,7:0,18:0,7:0,18:0
4,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,Las Vegas,NV,89118.0,36.063977,-115.241463,3.5,81,NaN,...,7:0,19:0,7:0,19:0,7:0,19:0,7:0,19:0,7:0,19:0


## Attributes

We have a ton of attribute columns, and likely don't need them all. Let's choose which ones to keep based off what might actually be useful so we can lower our dataset size.

Note: In the final table, I'll also include the JSON attributes dictionary so if we want to try doing things the NoSQL way we can (and if that works maybe we can delete the relational attribute columns from the db).

In [42]:
#First, get a list of just the columns with the 'attributes.' prefix. We can then use the indices of these rows to delete them all
#from the yelp table, and add back in only the attributes we want
attributes = yelp_flat.filter(regex='^attributes.',axis=1)

#Renaming the price range attribute to get rid of the weird 2
attributes['attributes.RestaurantsPriceRange'] = attributes['attributes.RestaurantsPriceRange2']


C:\Users\galla\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


We still have a similar problem as we had with Hours, where we need to differentiate between an attribute not being in a dictionary vs the dictionary itself being nonexistent for the business.

In [43]:
attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124891 entries, 0 to 124890
Data columns (total 40 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   attributes.Open24Hours                 11 non-null     object
 1   attributes.DietaryRestrictions         22 non-null     object
 2   attributes.RestaurantsCounterService   10 non-null     object
 3   attributes.AgesAllowed                 76 non-null     object
 4   attributes.BYOBCorkage                 851 non-null    object
 5   attributes.HairSpecializesIn           957 non-null    object
 6   attributes.DriveThru                   2983 non-null   object
 7   attributes.Smoking                     2567 non-null   object
 8   attributes.Music                       4859 non-null   object
 9   attributes.BestNights                  3673 non-null   object
 10  attributes.CoatCheck                   3142 non-null   object
 11  attributes.Ha

In [44]:
'''
Reminder of what our null_to_closed function looks like

def null_to_closed(series, thresh, replacement_string):
    # if at least one of the hours columns has a non null value, fill all the hour nulls with "Closed"
    if series.isna().sum() < thresh:
        series.fillna(replacement_string, inplace= True)
    
    return series
'''

attributes = attributes.apply(null_to_closed, args = (40, "Unlisted"), axis =1)
attributes.info()
attributes.head(40)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124891 entries, 0 to 124890
Data columns (total 40 columns):
 #   Column                                 Non-Null Count   Dtype 
---  ------                                 --------------   ----- 
 0   attributes.Open24Hours                 107063 non-null  object
 1   attributes.DietaryRestrictions         107063 non-null  object
 2   attributes.RestaurantsCounterService   107063 non-null  object
 3   attributes.AgesAllowed                 107063 non-null  object
 4   attributes.BYOBCorkage                 107063 non-null  object
 5   attributes.HairSpecializesIn           107063 non-null  object
 6   attributes.DriveThru                   107063 non-null  object
 7   attributes.Smoking                     107063 non-null  object
 8   attributes.Music                       107063 non-null  object
 9   attributes.BestNights                  107063 non-null  object
 10  attributes.CoatCheck                   107063 non-null  object
 11  

,attributes.Open24Hours,attributes.DietaryRestrictions,attributes.RestaurantsCounterService,attributes.AgesAllowed,attributes.BYOBCorkage,attributes.HairSpecializesIn,attributes.DriveThru,attributes.Smoking,attributes.Music,attributes.BestNights,...,attributes.RestaurantsTakeOut,attributes.RestaurantsAttire,attributes.WiFi,attributes.RestaurantsPriceRange2,attributes.ByAppointmentOnly,attributes.BusinessParking,attributes.GoodForKids,attributes.BikeParking,attributes.BusinessAcceptsCreditCards,attributes.RestaurantsPriceRange
0,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,Unlisted,Unlisted,Unlisted,3,False,"{'garage': False, 'street': False, 'validated'...",False,True,True,3
1,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,Unlisted,Unlisted,Unlisted,Unlisted,True,Unlisted,True,Unlisted,Unlisted,Unlisted
2,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,Unlisted,Unlisted,Unlisted,Unlisted,True,Unlisted,Unlisted,Unlisted,True,Unlisted
3,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,True,Unlisted
4,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,Unlisted,Unlisted,u'no',1,Unlisted,"{'garage': False, 'street': False, 'validated'...",Unlisted,True,True,1
5,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,Unlisted,Unlisted,Unlisted,Unlisted,False,Unlisted,Unlisted,Unlisted,True,Unlisted
6,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,True,u'casual',Unlisted,1,False,None,False,False,True,1
7,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,True,Unlisted
8,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,True,Unlisted,u'no',1,Unlisted,"{'garage': False, 'street': True, 'validated':...",Unlisted,True,True,1
9,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,...,Unlisted,Unlisted,Unlisted,Unlisted,True,Unlisted,Unlisted,Unlisted,True,Unlisted


In [45]:
#Make sure NaNs in rows where the dictionary did not exist were not overwritten
attributes[attributes.isna().any(axis=1)]

,attributes.Open24Hours,attributes.DietaryRestrictions,attributes.RestaurantsCounterService,attributes.AgesAllowed,attributes.BYOBCorkage,attributes.HairSpecializesIn,attributes.DriveThru,attributes.Smoking,attributes.Music,attributes.BestNights,...,attributes.RestaurantsTakeOut,attributes.RestaurantsAttire,attributes.WiFi,attributes.RestaurantsPriceRange2,attributes.ByAppointmentOnly,attributes.BusinessParking,attributes.GoodForKids,attributes.BikeParking,attributes.BusinessAcceptsCreditCards,attributes.RestaurantsPriceRange
34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:

#selecting desired attributes
desired_attributes = attributes[['attributes.HappyHour', 'attributes.GoodForDancing', 'attributes.WheelchairAccessible', 'attributes.RestaurantsDelivery', 'attributes.RestaurantsTakeOut', 'attributes.RestaurantsPriceRange', 'attributes.Alcohol', 'attributes.DogsAllowed', 'attributes.GoodForKids', 'attributes.BikeParking']]



In [47]:
desired_attributes.info()
desired_attributes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124891 entries, 0 to 124890
Data columns (total 10 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   attributes.HappyHour              107063 non-null  object
 1   attributes.GoodForDancing         107063 non-null  object
 2   attributes.WheelchairAccessible   107063 non-null  object
 3   attributes.RestaurantsDelivery    107063 non-null  object
 4   attributes.RestaurantsTakeOut     107063 non-null  object
 5   attributes.RestaurantsPriceRange  107063 non-null  object
 6   attributes.Alcohol                107063 non-null  object
 7   attributes.DogsAllowed            107063 non-null  object
 8   attributes.GoodForKids            107063 non-null  object
 9   attributes.BikeParking            107063 non-null  object
dtypes: object(10)
memory usage: 9.5+ MB


,attributes.HappyHour,attributes.GoodForDancing,attributes.WheelchairAccessible,attributes.RestaurantsDelivery,attributes.RestaurantsTakeOut,attributes.RestaurantsPriceRange,attributes.Alcohol,attributes.DogsAllowed,attributes.GoodForKids,attributes.BikeParking
0,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,3,Unlisted,Unlisted,False,True
1,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,True,Unlisted
2,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted
3,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted
4,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,1,Unlisted,Unlisted,Unlisted,True


There are some other potentially useful attributes that contain their own nested dictionaries. In order to not blow our table up too much with sparse data, we are going to include the full attributes json and and try to handle these through NoSQL for now. If we find they are popular queries, we will go back and expand them into relational columns. These are some of those attributes, along with their first non-null row, so we can get an idea of their keys.

In [48]:
#make another attributes to let us dirtily look at nested content
attributes2 = yelp_flat.filter(regex='^attributes.',axis=1)

In [49]:
att = attributes2['attributes.Music']
att.loc[att.first_valid_index()]

"{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': True, 'video': False, 'karaoke': False}"

In [50]:
att = attributes2['attributes.GoodForMeal']
att.loc[att.first_valid_index()]

"{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}"

In [51]:
att = attributes2['attributes.Ambience']
att.loc[att.first_valid_index()]

"{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': False}"

In [52]:
att = attributes2['attributes.BusinessParking']
att.loc[att.first_valid_index()]

"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"

In [53]:
#drop all attributes from original df, then add back just the desired ones
final_cols = [col for col in yelp_flat.columns if not col.startswith('attributes.')]
yelp_final = yelp_flat[final_cols]
yelp_final = pd.concat([yelp_final, desired_attributes], join='inner', axis=1)
yelp_final.info()
yelp_final.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124891 entries, 0 to 124890
Data columns (total 33 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   business_id                       124891 non-null  object 
 1   name                              124891 non-null  object 
 2   city                              124891 non-null  object 
 3   state                             124891 non-null  object 
 4   postal_code                       124891 non-null  float64
 5   latitude                          124891 non-null  float64
 6   longitude                         124891 non-null  float64
 7   stars                             124891 non-null  float64
 8   review_count                      124891 non-null  int64  
 9   hours.Sunday_open                 103793 non-null  object 
 10  hours.Sunday_closed               103793 non-null  object 
 11  hours.Saturday_open               103793 non-null  o

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,hours.Sunday_open,...,attributes.HappyHour,attributes.GoodForDancing,attributes.WheelchairAccessible,attributes.RestaurantsDelivery,attributes.RestaurantsTakeOut,attributes.RestaurantsPriceRange,attributes.Alcohol,attributes.DogsAllowed,attributes.GoodForKids,attributes.BikeParking
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,Cornelius,NC,28031.0,35.462724,-80.852612,3.5,36,13:0,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,3,Unlisted,Unlisted,False,True
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",Scottsdale,AZ,85258.0,33.569404,-111.890264,5.0,4,NaN,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,True,Unlisted
2,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,85205.0,33.428065,-111.726648,4.5,26,Closed,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted
3,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,Gilbert,AZ,85233.0,33.350399,-111.827142,4.5,38,Closed,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted
4,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,Las Vegas,NV,89118.0,36.063977,-115.241463,3.5,81,Closed,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,1,Unlisted,Unlisted,Unlisted,True


Of our included attributes, we have two that are not boolean (Alcohol and RestaurantsPriceRange). These are the values for those two:

In [54]:
att = yelp_final['attributes.Alcohol']
att.value_counts()

Unlisted            84860
u'none'              7605
u'full_bar'          6464
'none'               3297
u'beer_and_wine'     2089
'full_bar'           1820
'beer_and_wine'       912
None                   16
Name: attributes.Alcohol, dtype: int64

In [55]:
att = yelp_final['attributes.RestaurantsPriceRange']
att.value_counts()

Unlisted    52101
2           29969
1           19616
3            4263
4            1081
None           33
Name: attributes.RestaurantsPriceRange, dtype: int64

Price range is fine to leave as is (though we might want to include dollar signs in our front-end js), but Alcohol has an interesting phenomenon where there are duplicate row values with a 'u' prefix. We believe this is to signify that such a quality was determined by users rather than listed by the business. For our purposes, however, that seems a little to granular, so let's group user and business ratings together. There's also a 'none', which is likely a specific listing that no alcohol is offered, and a None, which might represent nothing being listed. However the latter only has 21 values, so for our sake it is probably safe to merge these as well.

In [56]:
yelp_final['attributes.Alcohol'].replace("u'none'", "None", inplace=True)
yelp_final['attributes.Alcohol'].replace("'none'", "None", inplace=True)
yelp_final['attributes.Alcohol'].replace("u'full_bar'", "full_bar", inplace=True)
yelp_final['attributes.Alcohol'].replace("u'beer_and_wine'", "beer_and_wine", inplace=True)
yelp_final['attributes.Alcohol'].replace("'full_bar'", "full_bar", inplace=True)
yelp_final['attributes.Alcohol'].replace("'beer_and_wine'", "beer_and_wine", inplace=True)

yelp_final['attributes.Alcohol'].value_counts()

Unlisted         84860
None             10918
full_bar          8284
beer_and_wine     3001
Name: attributes.Alcohol, dtype: int64

In [57]:
yelp_final = pd.concat([yelp_final,yelp_reduced['attributes']], join='inner', axis=1)

yelp_final.info()
yelp_final.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124891 entries, 0 to 124890
Data columns (total 34 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   business_id                       124891 non-null  object 
 1   name                              124891 non-null  object 
 2   city                              124891 non-null  object 
 3   state                             124891 non-null  object 
 4   postal_code                       124891 non-null  float64
 5   latitude                          124891 non-null  float64
 6   longitude                         124891 non-null  float64
 7   stars                             124891 non-null  float64
 8   review_count                      124891 non-null  int64  
 9   hours.Sunday_open                 103793 non-null  object 
 10  hours.Sunday_closed               103793 non-null  object 
 11  hours.Saturday_open               103793 non-null  o

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,hours.Sunday_open,...,attributes.GoodForDancing,attributes.WheelchairAccessible,attributes.RestaurantsDelivery,attributes.RestaurantsTakeOut,attributes.RestaurantsPriceRange,attributes.Alcohol,attributes.DogsAllowed,attributes.GoodForKids,attributes.BikeParking,attributes
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,Cornelius,NC,28031.0,35.462724,-80.852612,3.5,36,13:0,...,Unlisted,Unlisted,Unlisted,Unlisted,3,Unlisted,Unlisted,False,True,"{'BusinessAcceptsCreditCards': 'True', 'BikePa..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",Scottsdale,AZ,85258.0,33.569404,-111.890264,5.0,4,NaN,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,True,Unlisted,"{'GoodForKids': 'True', 'ByAppointmentOnly': '..."
2,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,85205.0,33.428065,-111.726648,4.5,26,Closed,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo..."
3,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,Gilbert,AZ,85233.0,33.350399,-111.827142,4.5,38,Closed,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,{'BusinessAcceptsCreditCards': 'True'}
4,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,Las Vegas,NV,89118.0,36.063977,-115.241463,3.5,81,Closed,...,Unlisted,Unlisted,Unlisted,Unlisted,1,Unlisted,Unlisted,Unlisted,True,"{'BusinessParking': '{'garage': False, 'street..."


In [156]:
yelp_final['attributes.RestaurantsPriceRange'].value_counts()

Unlisted    52101
2           29969
1           19616
3            4263
4            1081
None           33
Name: attributes.RestaurantsPriceRange, dtype: int64

In [58]:
yelp_final.to_csv('yelp_main.csv', index = False)

In [59]:
# No attributes-json included

yelp_final.drop('attributes', axis=1, inplace =True)


In [60]:
#Replace commas before saving to csv so they don't get misinterpreted
#After exploration, only 'name' has commas
yelp_final['name'] = yelp_final['name'].apply(lambda x: x.replace(',', '.'))

In [61]:
print(yelp_final[yelp_final['name'].str.contains(',')])

Empty DataFrame
Columns: [business_id, name, city, state, postal_code, latitude, longitude, stars, review_count, hours.Sunday_open, hours.Sunday_closed, hours.Saturday_open, hours.Saturday_closed, hours.Monday_open, hours.Monday_closed, hours.Tuesday_open, hours.Tuesday_closed, hours.Wednesday_open, hours.Wednesday_closed, hours.Thursday_open, hours.Thursday_closed, hours.Friday_open, hours.Friday_closed, attributes.HappyHour, attributes.GoodForDancing, attributes.WheelchairAccessible, attributes.RestaurantsDelivery, attributes.RestaurantsTakeOut, attributes.RestaurantsPriceRange, attributes.Alcohol, attributes.DogsAllowed, attributes.GoodForKids, attributes.BikeParking]
Index: []

[0 rows x 33 columns]


In [62]:
yelp_final.head()

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,hours.Sunday_open,...,attributes.HappyHour,attributes.GoodForDancing,attributes.WheelchairAccessible,attributes.RestaurantsDelivery,attributes.RestaurantsTakeOut,attributes.RestaurantsPriceRange,attributes.Alcohol,attributes.DogsAllowed,attributes.GoodForKids,attributes.BikeParking
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,Cornelius,NC,28031.0,35.462724,-80.852612,3.5,36,13:0,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,3,Unlisted,Unlisted,False,True
1,Yzvjg0SayhoZgCljUJRF9Q,Carlos Santo. NMD,Scottsdale,AZ,85258.0,33.569404,-111.890264,5.0,4,NaN,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,True,Unlisted
2,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,85205.0,33.428065,-111.726648,4.5,26,Closed,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted
3,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,Gilbert,AZ,85233.0,33.350399,-111.827142,4.5,38,Closed,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted
4,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,Las Vegas,NV,89118.0,36.063977,-115.241463,3.5,81,Closed,...,Unlisted,Unlisted,Unlisted,Unlisted,Unlisted,1,Unlisted,Unlisted,Unlisted,True


In [63]:
yelp_final.to_csv('yelp_main_nojson.csv', index = False)

## Categories

Now we'll deal with categories

In [114]:
#First, split comma separated string into list
categories['categories'] = categories['categories'].apply(lambda x: x.split(','))

We will have a grouping issue, since some words for each category string will start the string, while some will follow a comma and thus have a leading space. Let's strip all the tags of leading and trailing whitespace

In [128]:
def strip_tags(tag_list):
    return [tag.strip() for tag in tag_list]

In [133]:
categories['categories'] = categories['categories'].apply(lambda x: strip_tags(x))
categories.head()

,business_id,categories
0,f9NumwFMBDn751xgFiRbNA,"[Active Life, Gun/Rifle Ranges, Guns & Ammo, S..."
1,Yzvjg0SayhoZgCljUJRF9Q,"[Health & Medical, Fitness & Instruction, Yoga..."
2,51M2Kk903DFYI6gnB5I6SQ,"[Home Services, Plumbing, Electricians, Handym..."
3,cKyLV5oWZJ2NudWgqs8VZw,"[Auto Repair, Automotive, Oil Change Stations,..."
4,oiAlXZPIFm2nBCt0DHLu_Q,"[Dry Cleaning & Laundry, Local Services, Laund..."


In [134]:
#Explode categories into new rows
categories_exploded = categories.explode('categories')
categories_exploded.info()
categories_exploded.head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 543127 entries, 0 to 124891
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  543127 non-null  object
 1   categories   543127 non-null  object
dtypes: object(2)
memory usage: 12.4+ MB


,business_id,categories
0,f9NumwFMBDn751xgFiRbNA,Active Life
0,f9NumwFMBDn751xgFiRbNA,Gun/Rifle Ranges
0,f9NumwFMBDn751xgFiRbNA,Guns & Ammo
0,f9NumwFMBDn751xgFiRbNA,Shopping
1,Yzvjg0SayhoZgCljUJRF9Q,Health & Medical
1,Yzvjg0SayhoZgCljUJRF9Q,Fitness & Instruction
1,Yzvjg0SayhoZgCljUJRF9Q,Yoga
1,Yzvjg0SayhoZgCljUJRF9Q,Active Life
1,Yzvjg0SayhoZgCljUJRF9Q,Pilates
2,51M2Kk903DFYI6gnB5I6SQ,Home Services


In [135]:
categories_exploded['categories'].value_counts()

Restaurants            25765
Shopping               21495
Home Services          18841
Food                   15489
Health & Medical       15187
                       ...  
Ceremonial Clothing        1
Bungee Jumping             1
Speech Training            1
Street Art                 1
Bus Stations               1
Name: categories, Length: 1296, dtype: int64

In [136]:
categories_exploded.max()

business_id    zzwicjPC9g246MK2M1ZFBA
categories                       Zoos
dtype: object

In [137]:
categories_exploded.min()

business_id    --7zmmkVg-IMGaXbuVd0SQ
categories                 & Probates
dtype: object

In [138]:
categories_exploded.sort_values('categories')['categories'].unique()

array(['& Probates', '3D Printing', 'ATV Rentals/Tours', ..., 'Yoga',
       'Ziplining', 'Zoos'], dtype=object)

In [139]:
categories_exploded.sort_values('categories')['categories'].nunique()

1296

It looks like we have one funky tag: & Probates. Upon further inspection, it seems that this comes from the fact that one of the tags includes commas ('Wills, Trusts, & Probates'). If there were a ton of these, that might be cause for concern, as there is not really a regex function we could use to find all of those without knowing the number antecedents). Luckily, it's just the one though (as we could tell by sorting the values and seeing there are no more that start with an &). Let's combine those together

In [140]:
categories_exploded[categories_exploded.duplicated()]

,business_id,categories
3342,HEGy1__jKyMhkhXRW3O1ZQ,Gas Stations
3992,DppMDrrAE3eWkD3JE7gl7g,Gas Stations
5607,PSrITKG8En8mvwZ_YhywTg,Gas Stations
5890,nY9ZcdrkncyVh3ZJ4WvtwA,Gas Stations
8455,KoOqNG0ysV8yChCf5-XE0Q,Gas Stations
...,...,...
115220,0HDUNUvva3BEC1Yfe9an5A,Gas Stations
117143,mlkVAm2DkGnKnNVR1AnQLA,Gas Stations
117596,NQauHAQNg_a0z8WndFY3Wg,Gas Stations
120090,Mxn7K5xv7xXukrh10y8crg,Gas Stations


Somehow Gas Stations got duplicated- that's probably safe to just drop_duplicates

In [141]:
categories_exploded['categories'].replace('Wills', 'Wills, Trusts, & Probates', inplace=True)
categories_exploded['categories'].replace('Trusts', 'Wills, Trusts, & Probates', inplace=True)
categories_exploded['categories'].replace(' & Probates', 'Wills, Trusts, & Probates', inplace=True)

#delete duplicates
categories_exploded.drop_duplicates(inplace=True)

In [142]:
categories_exploded.sort_values('categories')['categories'].unique()

array(['& Probates', '3D Printing', 'ATV Rentals/Tours', ..., 'Yoga',
       'Ziplining', 'Zoos'], dtype=object)

In [143]:
categories_exploded[categories_exploded['categories'] == 'Wills, Trusts, & Probates']

,business_id,categories
2869,gFtYRB0AVTrakzNBgbq4nQ,"Wills, Trusts, & Probates"
6202,tNlfh4x5HUUHj65KbNmDFg,"Wills, Trusts, & Probates"
6354,GeVgX-OZDTlOskU9kd8TuA,"Wills, Trusts, & Probates"
6627,7RMJmOFSG3aSXU83KBoWGg,"Wills, Trusts, & Probates"
8580,BY9IQn3lvx6UGnPQQHAusQ,"Wills, Trusts, & Probates"
...,...,...
121126,fxkqaZgPUVrs1Tgdnrec1w,"Wills, Trusts, & Probates"
121229,T23Z6F4l0j-qriMSlyYTLA,"Wills, Trusts, & Probates"
121608,81H_yEsSDHcy2rXZ7EwsZw,"Wills, Trusts, & Probates"
122367,7YOEPQdMWGbKsat7xTeDwg,"Wills, Trusts, & Probates"


In [144]:
categories_exploded.duplicated().sum()

0

In [148]:
cat_set = set(categories_exploded['categories'])

In [150]:
#check out tags to see any blatant similarities
sorted(cat_set)

['& Probates',
 '3D Printing',
 'ATV Rentals/Tours',
 'Acai Bowls',
 'Accessories',
 'Accountants',
 'Acne Treatment',
 'Active Life',
 'Acupuncture',
 'Addiction Medicine',
 'Adoption Services',
 'Adult',
 'Adult Education',
 'Adult Entertainment',
 'Advertising',
 'Aerial Fitness',
 'Aerial Tours',
 'Aestheticians',
 'Afghan',
 'African',
 'Air Duct Cleaning',
 'Aircraft Dealers',
 'Aircraft Repairs',
 'Airlines',
 'Airport Lounges',
 'Airport Shuttles',
 'Airport Terminals',
 'Airports',
 'Airsoft',
 'Allergists',
 'Alternative Medicine',
 'Amateur Sports Teams',
 'American (New)',
 'American (Traditional)',
 'Amusement Parks',
 'Anesthesiologists',
 'Animal Assisted Therapy',
 'Animal Physical Therapy',
 'Animal Shelters',
 'Antiques',
 'Apartment Agents',
 'Apartments',
 'Appliances',
 'Appliances & Repair',
 'Appraisal Services',
 'Aquarium Services',
 'Aquariums',
 'Arabian',
 'Arcades',
 'Archery',
 'Architects',
 'Architectural Tours',
 'Argentine',
 'Armenian',
 'Art Classes'

In [151]:
categories_exploded.info()
categories_exploded.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 542936 entries, 0 to 124891
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  542936 non-null  object
 1   categories   542936 non-null  object
dtypes: object(2)
memory usage: 12.4+ MB


,business_id,categories
0,f9NumwFMBDn751xgFiRbNA,Active Life
0,f9NumwFMBDn751xgFiRbNA,Gun/Rifle Ranges
0,f9NumwFMBDn751xgFiRbNA,Guns & Ammo
0,f9NumwFMBDn751xgFiRbNA,Shopping
1,Yzvjg0SayhoZgCljUJRF9Q,Health & Medical


In [ ]:
'''
!pip3 install py_stringsimjoin
#!pip install python-Levenshtein
!pip3 install py_stringmatching
#!pip install difflib
'''

In [ ]:
'''import py_stringsimjoin as ssj
import py_stringmatching as sm
from Levenshtein import distance
from difflib import SequenceMatcher
'''

Let's see if we can find close matches for tags by using a Jaccard Join

In [ ]:
# create our tokenizer
#tokenizer = sm.QgramTokenizer(qval= 3, return_set= True )

In [146]:
# Replace commas in all fields to not mess up the csv file

#We'll replace the wills, trusts, & probates field we just made with other &s
categories_exploded.replace('Wills, Trusts, & Probates', 'Wills & Trusts & Probates', inplace =True)

#Check for any more commas
print(categories_exploded[categories_exploded['categories'].str.contains(',')])
print(categories_exploded[categories_exploded['business_id'].str.contains(',')])

Empty DataFrame
Columns: [business_id, categories]
Index: []
Empty DataFrame
Columns: [business_id, categories]
Index: []


In [157]:
categories_exploded[~categories_exploded['business_id'].isin(yelp_final['business_id'])]

,business_id,categories
29837,LiQckUkfrmaU09DijHLLpQ,Martial Arts
29837,LiQckUkfrmaU09DijHLLpQ,Fitness & Instruction
29837,LiQckUkfrmaU09DijHLLpQ,Active Life
29837,LiQckUkfrmaU09DijHLLpQ,Taekwondo


In [147]:
categories_exploded.to_csv('yelp_categories.csv', index= False)

In [158]:
yelp_reduced[yelp_reduced['business_id'] == 'LiQckUkfrmaU09DijHLLpQ']

,business_id,name,city,state,postal_code,latitude,longitude,stars,review_count,attributes,hours
29837,LiQckUkfrmaU09DijHLLpQ,NaN,Huntersville,NC,28078.0,35.383981,-80.786123,2.0,3,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","{'Monday': '10:0-20:30', 'Tuesday': '10:0-20:3..."
